# Initialize notebook

In [1]:
# Import libraries
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.image as mpimg
from scipy.optimize import fsolve
from math import sqrt
import time
from multiprocessing import Pool

In [2]:
# Constants
#KITTI_DATASET = '/media/javier/HDD_linux/KITTI_dataset/training/'
KITTI_DATASET = '/media/robesafe/SSD_SATA/KITTI_DATASET/'
#KITTI_DATASET = '/media/HDD_Linux/KITTI/training/'

# Load dataframe

In [3]:
df = pd.read_csv (KITTI_DATASET+'kitti_gt.csv')

In [4]:
df

,frame,id,type,truncated,occluded,alpha,left,top,right,bottom,height,width,length,x,y,z,ry,distance,height_image,width_image,theta_ray,incomplete_2d_horizontal,incomplete_2d_bottom
0,0,0,Pedestrian,0.0,0.0,-0.20,712.40,143.00,810.73,307.92,1.89,0.48,1.20,1.84,1.47,8.41,0.01,8.733533,164.92,98.33,0.21,False,False
1,1,0,Car,0.0,0.0,1.85,387.63,181.54,423.81,203.12,1.67,1.87,3.69,-16.53,2.39,58.49,1.57,60.827897,21.58,36.18,-0.28,False,False
2,1,1,Cyclist,0.0,3.0,-1.65,676.60,163.95,688.98,193.93,1.86,0.60,2.02,4.59,1.32,45.84,-1.55,46.088134,29.98,12.38,0.10,False,False
3,2,0,Car,0.0,0.0,-1.67,657.39,190.13,700.07,223.39,1.41,1.58,4.36,3.18,2.27,34.38,-1.58,34.601296,33.26,42.68,0.09,False,False
4,3,0,Car,0.0,0.0,1.55,614.24,181.78,727.31,284.77,1.57,1.73,4.15,1.00,1.75,13.22,1.62,13.372767,102.99,113.07,0.07,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34851,7480,9,Car,0.0,2.0,-0.33,749.78,170.88,917.51,243.48,1.77,1.69,3.82,5.72,1.75,18.85,-0.04,19.776334,72.60,167.73,0.29,False,False
34852,7480,10,Car,0.0,1.0,-2.90,344.07,180.05,479.10,224.15,1.49,1.59,4.44,-7.12,1.78,26.03,3.12,27.044846,44.10,135.03,-0.26,False,False
34853,7480,11,Car,0.0,2.0,-2.91,382.70,179.71,495.48,219.10,1.48,1.63,4.10,-6.79,1.78,28.78,-3.14,29.623654,39.39,112.78,-0.23,False,False
34854,7480,12,Car,0.0,0.0,-1.69,631.71,179.85,678.40,214.37,1.41,1.56,4.08,1.90,1.74,31.98,-1.63,32.083610,34.52,46.69,0.06,False,False


# 2D point projection to word coordinates

## Study the best method to obtain the inverse matrix

In [5]:
def get_intrinsic_matrix(image_id):
    name = '%06d'%image_id # 6 digit zeropadding
    with open(KITTI_DATASET+'calib/'+name+'.txt','r') as f:
        calib = f.readlines()

    # P2 (3 x 4) for left eye (intrinsic matrix in homogeneous coordinates)
    P2 = np.matrix([float(x) for x in calib[2].strip('\n').split(' ')[1:]]).reshape(3,4)
    return P2

In [6]:
def get_image_size(image_id):
    name = '%06d'%image_id # 6 digit zeropadding
    img = KITTI_DATASET+'images/'+name+'.png'
    
    # do projection staff
    png = mpimg.imread(img)
    IMG_H,IMG_W,_ = png.shape
    return (IMG_H, IMG_W)

In [7]:
def get_rectification_matrix(image_id):
    name = '%06d'%image_id # 6 digit zeropadding
    with open(KITTI_DATASET+'calib/'+name+'.txt','r') as f:
        calib = f.readlines()

    R0_rect = np.matrix([float(x) for x in calib[4].strip('\n').split(' ')[1:]]).reshape(3,3)
    # Add a 1 in bottom-right, reshape to 4 x 4
    R0_rect = np.insert(R0_rect,3,values=[0,0,0],axis=0)
    R0_rect = np.insert(R0_rect,3,values=[0,0,0,1],axis=1)
    return R0_rect

In [8]:
def get_translation_matrix(image_id):
    name = '%06d'%image_id # 6 digit zeropadding
    with open(KITTI_DATASET+'calib/'+name+'.txt','r') as f:
        calib = f.readlines()

    Tr_velo_to_cam = np.matrix([float(x) for x in calib[5].strip('\n').split(' ')[1:]]).reshape(3,4)
    Tr_velo_to_cam = np.insert(Tr_velo_to_cam,3,values=[0,0,0,1],axis=0)
    return Tr_velo_to_cam

In [9]:
# Get initial matrices and focal distance
image_id = 0

P2 = get_intrinsic_matrix(image_id)
R0_rect = get_rectification_matrix(image_id)
Tr_velo_to_cam = get_translation_matrix(image_id)
print("P2:"+str(P2))
print("R0_rect:"+str(R0_rect))
print("Tr_velo_to_cam:"+str(Tr_velo_to_cam))
f = P2[0,0]
print("f:"+str(f))
P = P2 * R0_rect * Tr_velo_to_cam
print("P:"+str(P))

P2:[[7.215377e+02 0.000000e+00 6.095593e+02 4.485728e+01]
 [0.000000e+00 7.215377e+02 1.728540e+02 2.163791e-01]
 [0.000000e+00 0.000000e+00 1.000000e+00 2.745884e-03]]
R0_rect:[[ 0.9999239   0.00983776 -0.00744505  0.        ]
 [-0.0098698   0.9999421  -0.00427846  0.        ]
 [ 0.00740253  0.00435161  0.9999631   0.        ]
 [ 0.          0.          0.          1.        ]]
Tr_velo_to_cam:[[ 7.533745e-03 -9.999714e-01 -6.166020e-04 -4.069766e-03]
 [ 1.480249e-02  7.280733e-04 -9.998902e-01 -7.631618e-02]
 [ 9.998621e-01  7.523790e-03  1.480755e-02 -2.717806e-01]
 [ 0.000000e+00  0.000000e+00  0.000000e+00  1.000000e+00]]
f:721.5377
P:[[ 6.09695409e+02 -7.21421597e+02 -1.25125855e+00 -1.23041806e+02]
 [ 1.80384202e+02  7.64479802e+00 -7.19651474e+02 -1.01016688e+02]
 [ 9.99945389e-01  1.24365378e-04  1.04513030e-02 -2.69386912e-01]]


In [10]:
# Get inverse matrix for each individual matrix
inv_P2 = np.linalg.pinv(P2)
inv_R0_rect = np.linalg.inv(R0_rect)
inv_Tr_velo_to_cam = np.linalg.inv(Tr_velo_to_cam)
print("inv_P2:"+str(inv_P2))
print("inv_R0_rect:"+str(inv_R0_rect))
print("inv_Tr_velo_to_cam:"+str(inv_Tr_velo_to_cam))
I_P2 = P2 * inv_P2
I_R0_rect = R0_rect * inv_R0_rect
I_Tr_velo_to_cam = Tr_velo_to_cam * inv_Tr_velo_to_cam
print("I_P2:"+str(I_P2))
print("I_R0_rect:"+str(I_R0_rect))
print("I_Tr_velo_to_cam:"+str(I_Tr_velo_to_cam))
inv_P_ = inv_Tr_velo_to_cam * inv_R0_rect * inv_P2
print("inv_P_:"+str(inv_P_))
I_P_ = P * inv_P_
print("I_P_:"+str(I_P_))

inv_P2:[[ 1.38098242e-03  2.95827322e-08 -8.41959541e-01]
 [ 2.95827322e-08  1.38592879e-03 -2.39580388e-01]
 [-2.26947721e-07  1.35725562e-09  1.00013059e+00]
 [ 8.26501486e-05 -4.94287311e-07 -4.75586646e-02]]
inv_R0_rect:[[ 0.99992388 -0.0098698   0.00740253  0.        ]
 [ 0.00983776  0.99994218  0.00435161  0.        ]
 [-0.00744505 -0.00427846  0.99996316  0.        ]
 [ 0.          0.          0.          1.        ]]
inv_Tr_velo_to_cam:[[ 7.53374455e-03  1.48024867e-02  9.99862010e-01  2.72903427e-01]
 [-9.99971462e-01  7.28073246e-04  7.52379023e-03 -1.96926586e-03]
 [-6.16602047e-04 -9.99890144e-01  1.48075511e-02 -7.22859005e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
I_P2:[[ 1.00000000e+00 -3.59040326e-17 -8.88178420e-14]
 [-7.04528124e-17  1.00000000e+00 -1.13686838e-13]
 [-2.17052193e-21  8.89918333e-19  1.00000000e+00]]
I_R0_rect:[[ 1.00000000e+00  1.73472348e-18  6.84402621e-19  0.00000000e+00]
 [-1.73472348e-18  1.00000000e+00  0.00000000e+

In [11]:
# Get inverse matrx of P
inv_P = np.linalg.pinv(P)
print("inv_P:"+str(inv_P))
I_inv_P = P * inv_P
print("I_inv_P:"+str(I_inv_P))

inv_P:[[ 2.00760422e-05 -1.16432598e-05  9.22431593e-01]
 [-1.38161998e-03  9.04536674e-06  8.26323262e-01]
 [-1.99071435e-05 -1.37880909e-03  2.78862481e-01]
 [ 7.31106956e-05 -9.67079297e-05 -2.76930824e-01]]
I_inv_P:[[ 1.00000000e+00  1.25767452e-16  1.13686838e-13]
 [ 1.04083409e-16  1.00000000e+00 -1.42108547e-13]
 [ 7.52165257e-19 -6.06475590e-19  1.00000000e+00]]


It is better to use the inverse matrices calculated one by one.

## Check the approximations of the 3D points

In [12]:
def get_P2R0(id_image):
    name = '%06d'%image_id # 6 digit zeropadding
    with open(KITTI_DATASET+'calib/'+name+'.txt','r') as f:
        calib = f.readlines()

    # P2 (3 x 4) for left eye (intrinsic matrix in homogeneous coordinates)
    P2 = np.matrix([float(x) for x in calib[2].strip('\n').split(' ')[1:]]).reshape(3,4)
    R0_rect = np.matrix([float(x) for x in calib[4].strip('\n').split(' ')[1:]]).reshape(3,3)
    # Add a 1 in bottom-right, reshape to 4 x 4
    R0_rect = np.insert(R0_rect,3,values=[0,0,0],axis=0)
    R0_rect = np.insert(R0_rect,3,values=[0,0,0,1],axis=1)
    
    return P2 * R0_rect

In [13]:
P2R0 = get_P2R0(0)
print(P2R0)

[[ 7.25995070e+02  9.75088151e+00  6.04164924e+02  4.48572800e+01]
 [-5.84187278e+00  7.22248117e+02  1.69760552e+02  2.16379100e-01]
 [ 7.40252700e-03  4.35161400e-03  9.99963100e-01  2.74588400e-03]]


In [14]:
P2

matrix([[7.215377e+02, 0.000000e+00, 6.095593e+02, 4.485728e+01],
        [0.000000e+00, 7.215377e+02, 1.728540e+02, 2.163791e-01],
        [0.000000e+00, 0.000000e+00, 1.000000e+00, 2.745884e-03]])

In [15]:
"""def equations(p):
    xc, yc, zc, w = p
    eq1 = (P2[0,0]*xc + P2[0,1]*yc + P2[0,2]*zc + P2[0,3])/ xi - w
    eq2 = (P2[1,0]*xc + P2[1,1]*yc + P2[1,2]*zc + P2[1,3])/ yi - w
    eq3 = (P2[2,0]*xc + P2[2,1]*yc + P2[2,2]*zc + P2[2,3]) - w
    eq4 = xc**2 + yc**2 + zc**2 - d**2
    return (eq1,eq2,eq3,eq4)"""

'def equations(p):\n    xc, yc, zc, w = p\n    eq1 = (P2[0,0]*xc + P2[0,1]*yc + P2[0,2]*zc + P2[0,3])/ xi - w\n    eq2 = (P2[1,0]*xc + P2[1,1]*yc + P2[1,2]*zc + P2[1,3])/ yi - w\n    eq3 = (P2[2,0]*xc + P2[2,1]*yc + P2[2,2]*zc + P2[2,3]) - w\n    eq4 = xc**2 + yc**2 + zc**2 - d**2\n    return (eq1,eq2,eq3,eq4)'

In [16]:
"""def equations(p):
    xc, yc, zc, w = p
    eq1 = (P2R0[0,0]*xc + P2R0[0,1]*yc + P2R0[0,2]*zc + P2R0[0,3])/ xi - w
    eq2 = (P2R0[1,0]*xc + P2R0[1,1]*yc + P2R0[1,2]*zc + P2R0[1,3])/ yi - w
    eq3 = (P2R0[2,0]*xc + P2R0[2,1]*yc + P2R0[2,2]*zc + P2R0[2,3]) - w
    eq4 = xc**2 + yc**2 + zc**2 - d**2
    return (eq1,eq2,eq3,eq4)"""

'def equations(p):\n    xc, yc, zc, w = p\n    eq1 = (P2R0[0,0]*xc + P2R0[0,1]*yc + P2R0[0,2]*zc + P2R0[0,3])/ xi - w\n    eq2 = (P2R0[1,0]*xc + P2R0[1,1]*yc + P2R0[1,2]*zc + P2R0[1,3])/ yi - w\n    eq3 = (P2R0[2,0]*xc + P2R0[2,1]*yc + P2R0[2,2]*zc + P2R0[2,3]) - w\n    eq4 = xc**2 + yc**2 + zc**2 - d**2\n    return (eq1,eq2,eq3,eq4)'

In [31]:
def solve_equation(q):  
    xi, yi, d = q
    def equations(p):
        xc, yc, zc, w = p
        eq1 = (P2[0,0]*xc + P2[0,1]*yc + P2[0,2]*zc + P2[0,3])/ xi - w
        eq2 = (P2[1,0]*xc + P2[1,1]*yc + P2[1,2]*zc + P2[1,3])/ yi - w
        eq3 = (P2[2,0]*xc + P2[2,1]*yc + P2[2,2]*zc + P2[2,3]) - w
        eq4 = xc**2 + yc**2 + zc**2 - d**2
        return (eq1,eq2,eq3,eq4)
    
    values = fsolve(equations, (0,0,0,0))
    print("\nError:")
    print(equations(values))
    return values

#point3D = np.array([5.72, 0.865, 18.85, 1])
point3D = np.array([-7.12, 1.78, 26.03, 1])
#point2D = [833.645, 207.18]
point2D = [411.585, 224.15]
print(point2D)
x_i = point2D[0]
y_i = point2D[1]
point3D_cam = R0_rect*Tr_velo_to_cam*point3D.reshape(4,1)
distance = float(np.sqrt(point3D[0]**2+point3D[1]**2+point3D[2]**2))

start = time.time()
#p = Pool(8)
#xd = list(p.map(solve_equation, [(100,200,20)]*4*10))
#p.close()
xd = list(map(solve_equation, [(201,283,5)]*20))
print(xd)
print(time.time()-start)

point2D_3D = solve_equation((x_i,y_i,distance))
point2D_3D = point2D_3D.reshape(4,1)
point2D_3D[3] = 1
print(point2D_3D)

[411.585, 224.15]

Error:
(-1.7763568394002505e-15, 0.0, 0.0, -1.3145040611561853e-13)

Error:
(-1.7763568394002505e-15, 0.0, 0.0, -1.3145040611561853e-13)

Error:
(-1.7763568394002505e-15, 0.0, 0.0, -1.3145040611561853e-13)

Error:
(-1.7763568394002505e-15, 0.0, 0.0, -1.3145040611561853e-13)

Error:
(-1.7763568394002505e-15, 0.0, 0.0, -1.3145040611561853e-13)

Error:
(-1.7763568394002505e-15, 0.0, 0.0, -1.3145040611561853e-13)

Error:
(-1.7763568394002505e-15, 0.0, 0.0, -1.3145040611561853e-13)

Error:
(-1.7763568394002505e-15, 0.0, 0.0, -1.3145040611561853e-13)

Error:
(-1.7763568394002505e-15, 0.0, 0.0, -1.3145040611561853e-13)

Error:
(-1.7763568394002505e-15, 0.0, 0.0, -1.3145040611561853e-13)

Error:
(-1.7763568394002505e-15, 0.0, 0.0, -1.3145040611561853e-13)

Error:
(-1.7763568394002505e-15, 0.0, 0.0, -1.3145040611561853e-13)

Error:
(-1.7763568394002505e-15, 0.0, 0.0, -1.3145040611561853e-13)

Error:
(-1.7763568394002505e-15, 0.0, 0.0, -1.3145040611561853e-13)

Error:
(-1.7763

## 2D point to 3D point conversion function in camera coordinates

In [18]:
class LocationSolver:
    def __init__(self, P2, inv_R0_rect, inv_Tr_velo_to_cam):
        self.P2 = P2
        self.inv_R0_rect = inv_R0_rect
        self.inv_Tr_velo_to_cam = inv_Tr_velo_to_cam
    
    def to_3d_camera(self, q):
        def equations(p):
            xc, yc, zc, w = p
            eq1 = (self.P2[0,0]*xc + self.P2[0,1]*yc + self.P2[0,2]*zc + self.P2[0,3])/ xi - w
            eq2 = (self.P2[1,0]*xc + self.P2[1,1]*yc + self.P2[1,2]*zc + self.P2[1,3])/ yi - w
            eq3 = (self.P2[2,0]*xc + self.P2[2,1]*yc + self.P2[2,2]*zc + self.P2[2,3]) - w
            eq4 = xc**2 + yc**2 + zc**2 - d**2
            return (eq1,eq2,eq3,eq4)

        xi, yi, d = q
        values = fsolve(equations, (0,0,0,0))
        values = values.reshape(4,1)
        values[3] = 1
        return values
    
    def to_3d_lidar(self, points3d_camera):
        points3d_lidar = inv_Tr_velo_to_cam * inv_R0_rect * points3d_camera
        points3d_lidar = np.squeeze(np.asarray(points3d_lidar))[:3]
        return points3d_lidar

In [19]:
def point2D_distance_to_point3D(points2d, location_solver):
    """Function for transforming a 2D point together with its distance to a 3D point
    input: points2d = [(x_i,y_i,distance)], P2
    output: points3d = [(x_c,y_c,z_c)]
    """
    pool = Pool(8)
    points3d = np.array(pool.map(location_solver.to_3d_camera, points2d))
    pool.close()
    return points3d

In [20]:
location_solver = LocationSolver(P2, inv_R0_rect, inv_Tr_velo_to_cam)
points3d_c = point2D_distance_to_point3D([(833.645, 207.18, 20),(411.585, 224.15, 30)], location_solver)
print(points3d_c)

[[[ 5.87189236]
  [ 0.90899673]
  [19.09697895]
  [ 1.        ]]

 [[-7.97573657]
  [ 2.05139867]
  [28.84751964]
  [ 1.        ]]]


## 3D camera to 3D LiDAR coordinate system

In [21]:
def point3dcamera_to_point3dlidar(points3d_c, location_solver):
    pool = Pool(8)
    points3d_l = np.array(pool.map(location_solver.to_3d_lidar, points3d_c))
    pool.close()
    return points3d_l

In [22]:
point3dcamera_to_point3dlidar(points3d_c, location_solver)

array([[19.37971602, -5.86155509, -0.84362154],
       [29.13841034,  7.99858366, -1.73771236]])

# Create pyradim planes

# Move pointcloud to the origin

In [23]:
PATH_TO_PCL = '/media/robesafe/SSD_SATA/KITTI_DATASET/velodyne/000000.bin'
PATH_TO_SAVE_NEW_PCL = '/home/robesafe/Javier/3D-detection-system-lidar-camera'

In [24]:
# Open pointcloud
pcl = np.fromfile(PATH_TO_PCL, dtype=np.float32).reshape(-1, 4)[:,:3]

In [25]:
pcl

array([[ 1.8324e+01,  4.9000e-02,  8.2900e-01],
       [ 1.8344e+01,  1.0600e-01,  8.2900e-01],
       [ 5.1299e+01,  5.0500e-01,  1.9440e+00],
       ...,
       [ 3.7180e+00, -1.4060e+00, -1.7370e+00],
       [ 3.7140e+00, -1.3910e+00, -1.7330e+00],
       [ 3.9670e+00, -1.4740e+00, -1.8570e+00]], dtype=float32)

In [26]:
def move_pcl_to_lidar(pcl, point3d):
    new_pcl = np.array(list(map(lambda point_pcl: point_pcl-point3d,pcl)))
    return new_pcl

In [27]:
move_pcl_to_lidar(pcl,np.array([19.37971602, -5.86155509, -0.84362154,]))

array([[ -1.05571662,   5.91055509,   1.67262154],
       [ -1.03571616,   5.96755509,   1.67262154],
       [ 31.91928377,   6.36655509,   2.78762155],
       ...,
       [-15.66171608,   4.45555507,  -0.89337845],
       [-15.66571603,   4.47055506,  -0.8893785 ],
       [-15.41271601,   4.38755511,  -1.01337845]])

## Example of moving 3D object to the origin of the PCL

In [28]:
location_solver = LocationSolver(P2, inv_R0_rect, inv_Tr_velo_to_cam)
points3d_c = point2D_distance_to_point3D([(397.065, 225.46, 8.733533)], location_solver)
points3d_l = point3dcamera_to_point3dlidar(points3d_c, location_solver)
print(points3d_l)

[[ 8.61899961  2.52239354 -0.5671274 ]]
